## Chapter 11: Logging and Error Handling

In any production application, things will go wrong. Users will encounter errors, external services will fail, and unexpected conditions will arise. How you handle these situations determines the reliability and maintainability of your system. **Logging** provides insight into what your application is doing, while **error handling** ensures that failures are managed gracefully and communicated clearly to clients. ASP.NET Core has built‑in, extensible logging and robust error handling middleware. In this chapter, you’ll learn how to use the `ILogger<T>` interface, configure logging providers, implement global exception handling, and create friendly error pages. By the end, you’ll be able to build applications that are observable and resilient.

### 11.1 Built‑in Logging Framework (`ILogger<T>`)

ASP.NET Core includes a logging API that works with a variety of built‑in and third‑party logging providers. The primary interface is `ILogger<T>`, where `T` is typically the class that is doing the logging. This ties log messages to the source class, making it easier to filter and analyze logs.

#### Creating a Logger

To use logging in a controller or service, inject `ILogger<T>` via the constructor:

```csharp
public class ProductsController : ControllerBase
{
    private readonly IProductService _productService;
    private readonly ILogger<ProductsController> _logger;

    public ProductsController(IProductService productService, ILogger<ProductsController> logger)
    {
        _productService = productService;
        _logger = logger;
    }

    [HttpGet("{id}")]
    public async Task<ActionResult<ProductDto>> GetProduct(int id)
    {
        _logger.LogInformation("Fetching product with ID {ProductId}", id);

        try
        {
            var product = await _productService.GetProductByIdAsync(id);
            if (product == null)
            {
                _logger.LogWarning("Product with ID {ProductId} not found", id);
                return NotFound();
            }

            return Ok(product);
        }
        catch (Exception ex)
        {
            _logger.LogError(ex, "Error fetching product with ID {ProductId}", id);
            throw; // Let global handler deal with it
        }
    }
}
```

#### Log Levels

Log messages have a level indicating their severity. The framework defines the following levels (from lowest to highest):

- `Trace` – Very detailed, potentially high‑volume logs, typically used only in development.
- `Debug` – Debugging information, less detailed than Trace.
- `Information` – General information about application flow.
- `Warning` – An unexpected but recoverable situation.
- `Error` – A failure that prevents a specific operation from completing.
- `Critical` – A catastrophic failure that may cause the application to abort.

You can filter logs based on these levels in configuration.

#### Log Message Structure

Always use **structured logging** (also called semantic logging). Instead of concatenating strings, use message templates with placeholders:

```csharp
_logger.LogInformation("Processing user {UserName} with ID {UserId}", user.Name, user.Id);
```

The placeholders (`{UserName}`, `{UserId}`) are not just for formatting; they become structured properties in the log output. This enables powerful querying and analysis in log aggregators like Seq, Elasticsearch, or Application Insights.

#### Available Logging Methods

The `ILogger` interface provides extension methods for each level:

- `LogTrace()`
- `LogDebug()`
- `LogInformation()`
- `LogWarning()`
- `LogError()`
- `LogCritical()`

Each accepts a message template and parameters, and optionally an exception.

### 11.2 Logging to Different Providers (Console, Debug, File, Seq)

By default, a new ASP.NET Core project includes the Console and Debug providers. You can add more providers via NuGet packages.

#### Default Configuration

In `Program.cs`, you’ll see:

```csharp
builder.Logging.ClearProviders();
builder.Logging.AddConsole();
builder.Logging.AddDebug();
```

`ClearProviders()` removes the default set (which may include Console, Debug, EventSource, etc.), then you explicitly add the ones you want. Alternatively, you can leave the defaults and add more.

#### Adding a File Provider

To log to a file, you can use a third‑party provider like **Serilog** (more on that later) or a simple file logger. For simplicity, we'll show how to add the built‑in file logging via the `Microsoft.Extensions.Logging.File` package (if available) but note that this is not part of the default templates; often people use Serilog for file logging.

**Using Serilog for file logging** (industry standard):

1. Install packages:
   ```bash
   dotnet add package Serilog.AspNetCore
   dotnet add package Serilog.Sinks.File
   ```

2. In `Program.cs`, configure Serilog before building the host:

```csharp
using Serilog;

Log.Logger = new LoggerConfiguration()
    .WriteTo.Console()
    .WriteTo.File("logs/myapp.txt", rollingInterval: RollingInterval.Day)
    .CreateLogger();

var builder = WebApplication.CreateBuilder(args);
builder.Host.UseSerilog(); // Replace default logging with Serilog
```

Now logs will go to both console and a daily rolling file.

#### Adding Seq

**Seq** is a structured log server. To send logs to Seq:

```bash
dotnet add package Serilog.Sinks.Seq
```

Then add the Seq sink:

```csharp
Log.Logger = new LoggerConfiguration()
    .WriteTo.Console()
    .WriteTo.Seq("http://localhost:5341")
    .CreateLogger();
```

#### Configuration via `appsettings.json`

You can control log levels and providers via configuration. For the built‑in logging, add a `Logging` section:

```json
{
  "Logging": {
    "LogLevel": {
      "Default": "Information",
      "Microsoft": "Warning",
      "Microsoft.Hosting.Lifetime": "Information"
    },
    "Console": {
      "LogLevel": {
        "Default": "Debug"
      }
    }
  }
}
```

This sets default levels for namespaces and overrides for the console provider.

### 11.3 Global Exception Handling Middleware

Unhandled exceptions in your application will crash the request and return a generic 500 error page (or no response in APIs). You need a centralized way to handle exceptions, log them, and return meaningful error responses.

#### Using `UseExceptionHandler` for MVC/Page Apps

For applications that return HTML, you can use the Exception Handler Middleware to redirect to a custom error page.

```csharp
if (!app.Environment.IsDevelopment())
{
    app.UseExceptionHandler("/Home/Error");
    app.UseHsts();
}
else
{
    app.UseDeveloperExceptionPage();
}
```

This registers a middleware that catches exceptions, logs them, and re‑executes the pipeline for the `/Home/Error` path. You need an `Error` action in your `HomeController`:

```csharp
[AllowAnonymous]
[ResponseCache(Duration = 0, Location = ResponseCacheLocation.None)]
public IActionResult Error()
{
    return View(new ErrorViewModel { RequestId = Activity.Current?.Id ?? HttpContext.TraceIdentifier });
}
```

The `Error.cshtml` view shows a friendly message.

#### Custom Exception Handling Middleware

For APIs, you typically want to return structured error responses. You can write custom middleware that catches exceptions and formats them as JSON.

```csharp
public class ErrorHandlingMiddleware
{
    private readonly RequestDelegate _next;
    private readonly ILogger<ErrorHandlingMiddleware> _logger;

    public ErrorHandlingMiddleware(RequestDelegate next, ILogger<ErrorHandlingMiddleware> logger)
    {
        _next = next;
        _logger = logger;
    }

    public async Task InvokeAsync(HttpContext context)
    {
        try
        {
            await _next(context);
        }
        catch (Exception ex)
        {
            _logger.LogError(ex, "An unhandled exception occurred.");
            await HandleExceptionAsync(context, ex);
        }
    }

    private static Task HandleExceptionAsync(HttpContext context, Exception exception)
    {
        context.Response.ContentType = "application/json";
        context.Response.StatusCode = (int)HttpStatusCode.InternalServerError;

        var response = new
        {
            error = new
            {
                message = "An error occurred while processing your request.",
                detail = exception.Message // Be careful: don't expose sensitive details in production
            }
        };

        return context.Response.WriteAsync(JsonSerializer.Serialize(response));
    }
}
```

Register it in `Program.cs` before other middleware:

```csharp
app.UseMiddleware<ErrorHandlingMiddleware>();
```

#### Using `UseExceptionHandler` with a Custom Handler for APIs

You can also use `UseExceptionHandler` with a lambda to return JSON:

```csharp
app.UseExceptionHandler(appError =>
{
    appError.Run(async context =>
    {
        context.Response.StatusCode = (int)HttpStatusCode.InternalServerError;
        context.Response.ContentType = "application/json";

        var contextFeature = context.Features.Get<IExceptionHandlerFeature>();
        if (contextFeature != null)
        {
            var logger = context.RequestServices.GetRequiredService<ILogger<Program>>();
            logger.LogError(contextFeature.Error, "Unhandled exception.");

            await context.Response.WriteAsync(JsonSerializer.Serialize(new
            {
                StatusCode = context.Response.StatusCode,
                Message = "Internal Server Error."
            }));
        }
    });
});
```

This keeps the built‑in exception handling but customizes the output.

### 11.4 Using `app.UseExceptionHandler` and Status Code Pages

#### Status Code Pages Middleware

Besides exceptions, you may want to handle non‑success status codes (e.g., 404 Not Found) in a friendly way. The Status Code Pages middleware can display custom pages for specific status codes.

```csharp
app.UseStatusCodePagesWithReExecute("/Home/Error/{0}");
```

Or for APIs, you can return JSON:

```csharp
app.UseStatusCodePages(async context =>
{
    context.HttpContext.Response.ContentType = "application/json";
    await context.HttpContext.Response.WriteAsync(JsonSerializer.Serialize(new
    {
        StatusCode = context.HttpContext.Response.StatusCode,
        Message = "The requested resource was not found."
    }));
});
```

#### Combining Exception and Status Code Handling

You can chain middleware:

```csharp
if (env.IsDevelopment())
{
    app.UseDeveloperExceptionPage();
}
else
{
    app.UseExceptionHandler("/error");
    app.UseStatusCodePagesWithReExecute("/error/{0}");
}
```

Then in an `ErrorController`:

```csharp
[Route("error")]
public class ErrorController : Controller
{
    [Route("{statusCode?}")]
    public IActionResult Index(int? statusCode)
    {
        if (statusCode.HasValue)
        {
            ViewBag.ErrorMessage = $"Status code: {statusCode}";
            // Log or handle specific status codes
        }
        return View();
    }
}
```

### 11.5 Structured Logging with Serilog (Advanced)

Structured logging goes beyond text; it captures properties that can be queried. Serilog is the most popular structured logging library for .NET.

#### Basic Setup with Serilog

As shown earlier, you configure Serilog at the program entry point.

```csharp
Log.Logger = new LoggerConfiguration()
    .MinimumLevel.Debug()
    .WriteTo.Console()
    .WriteTo.File("logs/myapp.txt", rollingInterval: RollingInterval.Day)
    .WriteTo.Seq("http://localhost:5341")
    .Enrich.WithProperty("Application", "MyApp")
    .Enrich.WithMachineName()
    .Enrich.WithThreadId()
    .CreateLogger();

try
{
    Log.Information("Starting web application");
    var builder = WebApplication.CreateBuilder(args);
    builder.Host.UseSerilog(); // Important: replaces ILoggerFactory
    // ... rest of builder
    var app = builder.Build();
    app.Run();
}
catch (Exception ex)
{
    Log.Fatal(ex, "Application start-up failed");
}
finally
{
    Log.CloseAndFlush();
}
```

Now all logs (including those from ASP.NET Core infrastructure) go through Serilog, capturing structured data.

#### Log Enrichment

Enrichers add additional properties to every log event, such as machine name, thread ID, or custom properties. This is invaluable for filtering in a central log system.

#### Destructuring

Serilog can destructure complex objects. For example, if you log a DTO, Serilog can serialize its properties.

```csharp
_logger.LogInformation("Created product {@Product}", productDto);
```

The `@` tells Serilog to serialize the object, not just call `ToString()`.

### 11.6 Health Checks Middleware (`/health` endpoints)

Production applications need a way to report their health to monitoring systems (like Kubernetes, Azure Load Balancer, etc.). ASP.NET Core provides health checks.

#### Adding Basic Health Checks

Install the package if needed (it's included in the web template):

```bash
dotnet add package Microsoft.AspNetCore.Diagnostics.HealthChecks
```

In `Program.cs`:

```csharp
builder.Services.AddHealthChecks();

// ...

app.MapHealthChecks("/health");
```

Now a GET request to `/health` returns a simple status (e.g., "Healthy").

#### Customizing Health Checks

You can add checks for specific dependencies:

```csharp
builder.Services.AddHealthChecks()
    .AddDbContextCheck<AppDbContext>()  // Checks database connectivity
    .AddUrlGroup(new Uri("https://api.example.com"), "External API");
```

You can also create custom health checks by implementing `IHealthCheck`.

#### Health Check UI

For a richer view, you can add the `AspNetCore.HealthChecks.UI` client, but that's beyond our scope.

### Summary

In this chapter, you’ve learned the essentials of logging and error handling in ASP.NET Core:

- **Logging** with `ILogger<T>` and structured message templates.
- Configuring **logging providers** (Console, File, Seq) and understanding log levels.
- Implementing **global exception handling** via middleware and built‑in exception handler.
- Using **status code pages** for friendly error responses.
- Advanced logging with **Serilog** for structured, queryable logs.
- Adding **health checks** for monitoring application availability.

With these tools, your applications become observable and resilient, capable of surviving failures and providing insights into their behavior.

**Exercise:**

1. In your existing project, add logging to all controller actions (Information for normal flow, Warning for missing resources, Error for exceptions).
2. Configure Serilog to log to both console and a rolling file. Set minimum log level to Debug in development and Warning in production (use `appsettings.json` for configuration).
3. Create a custom exception handling middleware that catches exceptions and returns a JSON error response for API controllers (check if the request expects JSON via `Accept` header).
4. Add a health check endpoint that verifies database connectivity (using `AddDbContextCheck`).
5. Test your error handling by causing an exception (e.g., divide by zero) and verify that it's logged and a proper response is returned.

In the next chapter, **"Configuration and Options Pattern,"** you’ll learn how to manage application settings across environments, bind configuration to strongly‑typed classes, and use the options pattern for clean, testable configuration access.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../2. core_concepts_data_access/10. web_apis_with_aspnet_core.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='12. configuration_and_options_pattern.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
